In [1]:
import lib.RayTracer as RT
import math
import numpy as np
import time
from PIL import Image
from stl import mesh

#define render window size
(w, h) = (640, 480)

Vec4| x: [1.] y: [0.] z: [0.] t: [0.]
Vec4| x: [1.66666667] y: [0.] z: [0.] t: [-1.33333333]
Vec4| x: [1.] y: [0.] z: [0.] t: [0.]
Vec4| x: [1.66666667] y: [0.] z: [0.] t: [-1.33333333]
Vec4| x: [-1.] y: [0.] z: [0.] t: [0.]
Vec4| x: [1.1785113] y: [1.1785113] z: [0.] t: [-1.33333333]
Vec4| x: [0.70710678] y: [-0.70710678] z: [0.] t: [0.]
Vec3| x: [-0.6] y: [0.8] z: [0.]


# Test raytracer

In [2]:
# Using an existing stl file:
your_mesh = mesh.Mesh.from_file('models/block100.stl')
your_mesh.v0

array([[  0., 100., 100.],
       [  0.,   0., 100.],
       [100., 100., 100.],
       [100.,   0., 100.],
       [100., 100.,   0.],
       [100.,   0.,   0.],
       [  0., 100.,   0.],
       [  0.,   0.,   0.],
       [100., 100., 100.],
       [  0., 100., 100.],
       [100.,   0.,   0.],
       [  0.,   0.,   0.]], dtype=float32)

In [3]:
#     Sphere(vec3(.75, .1, 1), .6, rgb(0, 0, 1)),
#     Sphere(vec3(-.75, .1, 2.25), .6, rgb(.5, .22, .5)),
#     Sphere(vec3(-2.75, .1, 3.5), .6, rgb(1, .572, .184)),
# 	Triangle(vec3(-0.3, -0.1, 1), vec3(-0.4, .5, .5), vec3(-0.35, .5, 2), None, rgb(1, 0.6, 0), 0.9),
# 	Triangle(vec3(-0.3, -0.1, 1), vec3(-0.35, .5, 2), vec3(3, 0.4, 2.5), None, rgb(1, 1, 1), 0.9),
# CheckeredSphere(vec3(0, -99999.5, 0), 99999, rgb(.75, .75, .75), 0.25),


scene = [
	RT.Sphere(RT.vec3(-5, .1, 6), .6, RT.rgb(0, 0, 1), beta=RT.velo(.99,0,0)),
	RT.Sphere(RT.vec3(-2.75, .1, 3.5), .6, RT.rgb(1, .572, .184)),
	RT.CheckeredSphere(RT.vec3(0, -99999.5, 0), 99999, RT.rgb(.75, .75, .75), 0.25),
	RT.Mesh(RT.vec3(-10, 40, 120),
	     mesh.Mesh.rotation_matrix(np.array([1, 0, 0]), math.radians(0)),
	     your_mesh,
	     RT.rgb(.7, .8, .7), 0.5)
]

# unpack scene (mesh into triangles)
offset = 0
for i in range(len(scene)):
	if scene[i].type == 2:
		for t in scene[i].tArray:
			scene.insert(i, t)
			offset += 1
		scene.pop(i+offset)
print("finished unpacking")
print(scene)
# finish unpacking

r = float(w) / h
# Screen coordinates: x0, y0, x1, y1.
S = (-1, 1 / r + .25, 1, -1 / r + .25)
x = np.tile(np.linspace(S[0], S[2], w), h)
y = np.repeat(np.linspace(S[1], S[3], h), w)

t0 = time.time()
Q = RT.vec3(x, y, 0)
color = RT.raytrace(RT.E, (Q - RT.E).norm(), scene)
print ("Took", time.time() - t0)

rgb = [Image.fromarray((255 * np.clip(c, 0, 1).reshape((h, w))).astype(np.uint8), "L") for c in color.components()]
display(Image.merge("RGB", rgb))

Created mesh.
finished unpacking
[<lib.RayTracer.Sphere object at 0x1158e4370>, <lib.RayTracer.Sphere object at 0x1158e4310>, <lib.RayTracer.CheckeredSphere object at 0x1158e4160>, <lib.RayTracer.Triangle object at 0x115887f10>, <lib.RayTracer.Triangle object at 0x115886fb0>, <lib.RayTracer.Triangle object at 0x115886e00>, <lib.RayTracer.Triangle object at 0x115886c50>, <lib.RayTracer.Triangle object at 0x115885ea0>, <lib.RayTracer.Triangle object at 0x115887850>, <lib.RayTracer.Triangle object at 0x1158876a0>, <lib.RayTracer.Triangle object at 0x1158874f0>, <lib.RayTracer.Triangle object at 0x115886290>, <lib.RayTracer.Triangle object at 0x1158858a0>, <lib.RayTracer.Triangle object at 0x1158e4d30>, <lib.RayTracer.Triangle object at 0x1158e7790>]


TypeError: unsupported operand type(s) for -: 'float' and 'vec3'